In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
from pavlov import stats
ax = stats.plot(['*perky-boxes*', '*forked-shots', '*better-slopes', -1], 'elo-mohex', 'μ', skip=3, fill=True, head=720)
ax.set_ylim(None, 0)

In [ ]:
from boardlaw.analysis import *
smoothed = grad_student_descent()

In [ ]:
boardsize = 9
with plt.style.context('seaborn-poster'):
    colors = ['C7']*(smoothed.shape[1]-1) + ['C1']
    ax = smoothed.plot(color=colors, legend=False, linewidth=1.5)
    ax.set_facecolor('whitesmoke')
    ax.grid(axis='y')
    ax.set_ylim(None, 0)
    ax.set_ylabel('eElo')
    ax.set_title(f'grad student ascent, {boardsize}x{boardsize}')
    

In [ ]:
stats.review()